# Iron Ore Indicator - P&L Analysis

**Date**: 2025-11-04
**Instrument**: DCE/i<00> (Iron Ore)
**Period**: 2024-10-01 to 2024-12-31 (3 months)
**Purpose**: Backtest performance analysis with real svr3 data

## 1. Setup and Imports

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import asyncio
import svr3
from datetime import datetime

%matplotlib inline
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (15, 10)

# Jupyter async support
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("✓ nest_asyncio enabled")
except ImportError:
    print("⚠️ Install nest-asyncio: pip install nest-asyncio")

# Load environment
from dotenv import load_dotenv
load_dotenv()

SVR_HOST = os.getenv("SVR_HOST")
SVR_TOKEN = os.getenv("SVR_TOKEN")

if not SVR_HOST or not SVR_TOKEN:
    print("❌ Error: SVR_HOST or SVR_TOKEN not found in .env file")
else:
    print(f"✓ Environment loaded: {SVR_HOST}")

✓ nest_asyncio enabled
✓ Environment loaded: 10.99.100.116


## 2. Data Fetching from SVR3 Server

In [2]:
async def fetch_indicator_data(start_date, end_date):
    """
    Fetch indicator data from svr3 server.
    
    Args:
        start_date: int format YYYYMMDDHHMMSS
        end_date: int format YYYYMMDDHHMMSS
    """
    print(f"📥 Fetching data from {start_date} to {end_date}...")
    
    # Server configuration
    RAILS_URL = f"https://{SVR_HOST}:4433/private-api/"
    WS_URL = f"wss://{SVR_HOST}:4433/tm"
    TM_MASTER = (SVR_HOST, 6102)
    
    try:
        # Create reader (CRITICAL: exact argument order)
        reader = svr3.sv_reader(
            start_date,          # start
            end_date,            # end
            "IronOreIndicator",  # algoname
            900,                 # granularity
            "private",           # namespace
            "symbol",            # work_mode
            ["DCE"],             # markets
            ["i<00>"],           # codes
            False,               # persistent
            RAILS_URL,           # rails_url
            WS_URL,              # ws_url
            "",                  # user
            "",                  # hashed_password
            TM_MASTER            # tm_master
        )
        
        # Set token
        reader.token = SVR_TOKEN
        
        # Connection sequence (DO NOT CHANGE ORDER)
        print("   🔐 Logging in...")
        await reader.login()
        
        print("   🔌 Connecting...")
        await reader.connect()
        
        print("   🔄 Starting WebSocket loop...")
        reader.ws_task = asyncio.create_task(reader.ws_loop())
        
        print("   🤝 Handshaking...")
        await reader.shakehand()
        
        print("   📊 Fetching data...")
        ret = await reader.save_by_symbol()
        
        # Extract data (structure: ret[1][1])
        data = ret[1][1]
        
        if not data or len(data) == 0:
            print("   ⚠️ No data returned")
            return None
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        
        # Parse timestamps if available
        if 'time_tag' in df.columns:
            try:
                import pycaitlynutils3 as pcu3
                df['timestamp'] = pd.to_datetime(df['time_tag'].apply(pcu3.ts_parse))
            except:
                # Fallback
                df['timestamp'] = pd.to_datetime(df['time_tag'], unit='ms')
        
        print(f"   ✓ Fetched {len(df)} bars")
        return df
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None

# Fetch data for 3-month period
start_date = 20241001000000  # 2024-10-01
end_date = 20241231235959    # 2024-12-31

df = await fetch_indicator_data(start_date, end_date)

if df is not None:
    print(f"\n✅ Data loaded successfully!")
    print(f"   Rows: {len(df)}")
    print(f"   Columns: {list(df.columns)}")
    print(f"   Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
    display(df.head())
else:
    print("\n❌ Failed to load data")

📥 Fetching data from 20241001000000 to 20241231235959...
   🔐 Logging in...
   🔌 Connecting...
   🔄 Starting WebSocket loop...
   🤝 Handshaking...
   📊 Fetching data...
   ❌ Error: cannot unpack non-iterable NoneType object

❌ Failed to load data


Traceback (most recent call last):
  File "/tmp/ipykernel_10159/1721714328.py", line 52, in fetch_indicator_data
    ret = await reader.save_by_symbol()
  File "/home/wolverine/bin/running/svr3.py", line 85, in save_by_symbol
    fields, stat = await self.sv_by_symbol(
TypeError: cannot unpack non-iterable NoneType object


## 3. P&L Calculation

In [3]:
def calculate_trades(df, entry_threshold=0.6, exit_threshold=0.3):
    """
    Calculate trades from indicator signals.
    
    Args:
        df: DataFrame with columns [timestamp, close, signal, confidence]
        entry_threshold: Minimum confidence to enter position
        exit_threshold: Minimum confidence to stay in position
    
    Returns:
        DataFrame with trade details
    """
    trades = []
    position = None
    
    for idx, row in df.iterrows():
        signal = row['signal']
        confidence = row['confidence']
        close = row['close']
        timestamp = row['timestamp']
        
        # Open position
        if position is None and signal != 0 and confidence > entry_threshold:
            position = {
                'entry_time': timestamp,
                'entry_price': close,
                'direction': signal,  # 1 for buy, -1 for sell
                'entry_confidence': confidence
            }
        
        # Close position
        elif position is not None:
            # Exit conditions:
            # 1. Signal reverses
            # 2. Confidence drops below threshold
            if (signal * position['direction'] < 0) or (confidence < exit_threshold):
                # Calculate P&L
                pnl = (close - position['entry_price']) * position['direction']
                
                trades.append({
                    'entry_time': position['entry_time'],
                    'exit_time': timestamp,
                    'entry_price': position['entry_price'],
                    'exit_price': close,
                    'direction': position['direction'],
                    'pnl': pnl,
                    'return_pct': pnl / position['entry_price'] * 100
                })
                
                position = None
    
    return pd.DataFrame(trades)

def calculate_cumulative_pnl(df, trades):
    """
    Calculate cumulative P&L at each timestamp.
    """
    df['cumulative_pnl'] = 0.0
    
    cumulative = 0.0
    trade_idx = 0
    
    for idx, row in df.iterrows():
        timestamp = row['timestamp']
        
        # Add completed trades up to this timestamp
        while trade_idx < len(trades) and trades.iloc[trade_idx]['exit_time'] <= timestamp:
            cumulative += trades.iloc[trade_idx]['pnl']
            trade_idx += 1
        
        df.at[idx, 'cumulative_pnl'] = cumulative
    
    return df

def calculate_drawdown(cumulative_pnl):
    """
    Calculate drawdown from peak.
    """
    peak = cumulative_pnl.expanding().max()
    drawdown = (cumulative_pnl - peak) / peak.replace(0, 1)  # Avoid division by zero
    return drawdown

# Calculate trades
print("📊 Calculating trades...")
trades = calculate_trades(df)
print(f"   Total trades: {len(trades)}")

if len(trades) > 0:
    # Calculate cumulative P&L
    df = calculate_cumulative_pnl(df, trades)
    df['drawdown'] = calculate_drawdown(df['cumulative_pnl'])
    
    print(f"   Total P&L: ¥{trades['pnl'].sum():,.2f}")
    print(f"   Winning trades: {(trades['pnl'] > 0).sum()}")
    print(f"   Losing trades: {(trades['pnl'] < 0).sum()}")
    
    # Display sample trades
    display(trades.head(10))
else:
    print("   ⚠️ No trades generated")

📊 Calculating trades...


AttributeError: 'NoneType' object has no attribute 'iterrows'

## 4. Performance Metrics

In [ ]:
def calculate_metrics(df, trades):
    """
    Calculate comprehensive performance metrics.
    """
    if len(trades) == 0:
        return None
    
    winning_trades = trades[trades['pnl'] > 0]
    losing_trades = trades[trades['pnl'] < 0]
    
    metrics = {
        'Total Trades': len(trades),
        'Winning Trades': len(winning_trades),
        'Losing Trades': len(losing_trades),
        'Win Rate': len(winning_trades) / len(trades) if len(trades) > 0 else 0,
        'Total P&L (CNY)': trades['pnl'].sum(),
        'Avg Win (CNY)': winning_trades['pnl'].mean() if len(winning_trades) > 0 else 0,
        'Avg Loss (CNY)': losing_trades['pnl'].mean() if len(losing_trades) > 0 else 0,
        'Largest Win (CNY)': winning_trades['pnl'].max() if len(winning_trades) > 0 else 0,
        'Largest Loss (CNY)': losing_trades['pnl'].min() if len(losing_trades) > 0 else 0,
        'Profit Factor': (winning_trades['pnl'].sum() / abs(losing_trades['pnl'].sum())) 
                        if len(losing_trades) > 0 and losing_trades['pnl'].sum() != 0 else np.inf,
        'Sharpe Ratio': (trades['pnl'].mean() / trades['pnl'].std() * np.sqrt(252)) 
                       if trades['pnl'].std() > 0 else 0,
        'Max Drawdown': df['drawdown'].min(),
        'Max Drawdown (%)': df['drawdown'].min() * 100
    }
    
    return metrics

# Calculate and display metrics
metrics = calculate_metrics(df, trades)

if metrics:
    print("\n" + "="*60)
    print("PERFORMANCE METRICS")
    print("="*60)
    
    metrics_df = pd.DataFrame([metrics]).T
    metrics_df.columns = ['Value']
    
    # Format specific rows
    for idx in metrics_df.index:
        if 'Rate' in idx or 'Drawdown (%)' in idx:
            metrics_df.loc[idx, 'Value'] = f"{metrics_df.loc[idx, 'Value']:.2%}"
        elif 'CNY' in idx:
            metrics_df.loc[idx, 'Value'] = f"¥{metrics_df.loc[idx, 'Value']:,.2f}"
        elif idx in ['Sharpe Ratio', 'Profit Factor']:
            metrics_df.loc[idx, 'Value'] = f"{metrics_df.loc[idx, 'Value']:.2f}"
    
    display(metrics_df)
    print("="*60)
else:
    print("⚠️ Cannot calculate metrics (no trades)")

## 5. P&L Visualization (4 Panels)

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(15, 12))

# Panel 1: Cumulative P&L
axes[0].plot(df['timestamp'], df['cumulative_pnl'], linewidth=2, color='blue')
axes[0].set_title('Cumulative P&L', fontsize=14, fontweight='bold')
axes[0].set_ylabel('P&L (CNY)', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='black', linestyle='--', alpha=0.5)

# Add final P&L annotation
final_pnl = df['cumulative_pnl'].iloc[-1]
axes[0].text(0.02, 0.95, f'Final P&L: ¥{final_pnl:,.2f}', 
            transform=axes[0].transAxes, fontsize=10, 
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Panel 2: Price and Signals
buy_signals = df[df['signal'] == 1]
sell_signals = df[df['signal'] == -1]

axes[1].plot(df['timestamp'], df['close'], alpha=0.6, color='gray', label='Price', linewidth=1.5)
axes[1].scatter(buy_signals['timestamp'], buy_signals['close'],
               color='green', marker='^', s=100, label='Buy', alpha=0.8, zorder=5)
axes[1].scatter(sell_signals['timestamp'], sell_signals['close'],
               color='red', marker='v', s=100, label='Sell', alpha=0.8, zorder=5)
axes[1].set_title('Price and Trading Signals', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Price (CNY)', fontsize=12)
axes[1].legend(loc='upper left')
axes[1].grid(True, alpha=0.3)

# Panel 3: Drawdown
axes[2].fill_between(df['timestamp'], 0, df['drawdown'] * 100,
                     color='red', alpha=0.3)
axes[2].plot(df['timestamp'], df['drawdown'] * 100, color='darkred', linewidth=1)
axes[2].set_title('Drawdown', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Drawdown (%)', fontsize=12)
axes[2].grid(True, alpha=0.3)

# Add max drawdown annotation
max_dd = df['drawdown'].min() * 100
axes[2].text(0.02, 0.05, f'Max Drawdown: {max_dd:.2f}%', 
            transform=axes[2].transAxes, fontsize=10,
            verticalalignment='bottom',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Panel 4: Trade P&L Distribution
if len(trades) > 0:
    axes[3].hist(trades['pnl'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    axes[3].axvline(x=0, color='black', linestyle='--', linewidth=2, label='Break-even')
    axes[3].axvline(x=trades['pnl'].mean(), color='red', linestyle='-', linewidth=2, label='Mean')
    axes[3].set_title('Individual Trade P&L Distribution', fontsize=14, fontweight='bold')
    axes[3].set_xlabel('P&L (CNY)', fontsize=12)
    axes[3].set_ylabel('Frequency', fontsize=12)
    axes[3].legend()
    axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('pnl_analysis.png', dpi=150, bbox_inches='tight')
print("\n✅ Visualization saved as 'pnl_analysis.png'")
plt.show()

## 6. Signal Analysis

In [ ]:
# Analyze signal characteristics
print("📊 Signal Analysis:")
print("="*60)

signal_df = df[df['signal'] != 0].copy()

if len(signal_df) > 0:
    signal_analysis = signal_df.groupby('signal').agg({
        'confidence': ['count', 'mean', 'std', 'min', 'max'],
        'rsi': ['mean', 'std'],
        'ema_fast': 'mean',
        'ema_slow': 'mean'
    }).round(4)
    
    signal_analysis.index = signal_analysis.index.map({-1: 'SELL', 1: 'BUY'})
    
    display(signal_analysis)
    
    # Signal distribution
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Confidence distribution by signal
    buy_conf = signal_df[signal_df['signal'] == 1]['confidence']
    sell_conf = signal_df[signal_df['signal'] == -1]['confidence']
    
    axes[0].hist([buy_conf, sell_conf], bins=20, label=['Buy', 'Sell'], 
                color=['green', 'red'], alpha=0.7, edgecolor='black')
    axes[0].set_title('Signal Confidence Distribution', fontweight='bold')
    axes[0].set_xlabel('Confidence')
    axes[0].set_ylabel('Frequency')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # RSI distribution by signal
    buy_rsi = signal_df[signal_df['signal'] == 1]['rsi']
    sell_rsi = signal_df[signal_df['signal'] == -1]['rsi']
    
    axes[1].hist([buy_rsi, sell_rsi], bins=20, label=['Buy', 'Sell'], 
                color=['green', 'red'], alpha=0.7, edgecolor='black')
    axes[1].set_title('RSI at Signal Time', fontweight='bold')
    axes[1].set_xlabel('RSI')
    axes[1].set_ylabel('Frequency')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No signals to analyze")

## 7. Monthly Performance Breakdown

In [ ]:
# Monthly P&L breakdown
print("📅 Monthly Performance:")
print("="*60)

df['month'] = pd.to_datetime(df['timestamp']).dt.to_period('M')
monthly_pnl = df.groupby('month')['cumulative_pnl'].last().diff()

# Handle first month (no diff)
first_month = df.groupby('month')['cumulative_pnl'].last().iloc[0]
monthly_pnl.iloc[0] = first_month

print(monthly_pnl)
print(f"\nTotal: ¥{monthly_pnl.sum():,.2f}")

# Visualization
fig, ax = plt.subplots(figsize=(12, 6))
colors = ['green' if x > 0 else 'red' for x in monthly_pnl]
monthly_pnl.plot(kind='bar', ax=ax, color=colors, edgecolor='black')
ax.set_title('Monthly P&L', fontsize=14, fontweight='bold')
ax.set_ylabel('P&L (CNY)', fontsize=12)
ax.set_xlabel('Month', fontsize=12)
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45)

# Add value labels on bars
for i, v in enumerate(monthly_pnl):
    ax.text(i, v, f'¥{v:,.0f}', ha='center', va='bottom' if v > 0 else 'top', fontsize=9)

plt.tight_layout()
plt.show()

## 8. Summary and Recommendations

In [ ]:
print("\n" + "="*60)
print("SUMMARY")
print("="*60)

if metrics:
    print(f"Period: {df['timestamp'].min().date()} to {df['timestamp'].max().date()}")
    print(f"Total Bars: {len(df)}")
    print(f"Total Trades: {metrics['Total Trades']}")
    print(f"Total P&L: ¥{metrics['Total P&L (CNY)']:,.2f}")
    print(f"Win Rate: {metrics['Win Rate']:.2%}")
    print(f"Sharpe Ratio: {metrics['Sharpe Ratio']:.2f}")
    print(f"Max Drawdown: {metrics['Max Drawdown (%)']:.2f}%")
    
    print("\n" + "="*60)
    print("EVALUATION")
    print("="*60)
    
    # Evaluate against criteria
    criteria = [
        ("✅" if metrics['Sharpe Ratio'] > 1.0 else "❌", 
         f"Sharpe Ratio > 1.0: {metrics['Sharpe Ratio']:.2f}"),
        ("✅" if metrics['Win Rate'] > 0.5 else "❌", 
         f"Win Rate > 50%: {metrics['Win Rate']:.2%}"),
        ("✅" if metrics['Profit Factor'] > 1.5 else "❌", 
         f"Profit Factor > 1.5: {metrics['Profit Factor']:.2f}"),
        ("✅" if abs(metrics['Max Drawdown']) < 0.2 else "❌", 
         f"Max Drawdown < 20%: {abs(metrics['Max Drawdown (%)']):.2f}%"),
        ("✅" if metrics['Total P&L (CNY)'] > 0 else "❌", 
         f"Total P&L > 0: ¥{metrics['Total P&L (CNY)']:,.2f}")
    ]
    
    for status, criterion in criteria:
        print(f"{status} {criterion}")
    
    passed = sum(1 for status, _ in criteria if status == "✅")
    print(f"\n📊 Passed: {passed}/{len(criteria)} criteria")
    
    if passed == len(criteria):
        print("\n🎉 All acceptance criteria met!")
    else:
        print("\n⚠️ Some criteria not met. Consider parameter optimization.")
else:
    print("❌ No metrics available")

print("="*60)